<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/gemma_ft_dolly__with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#  This notebook demonstrates the following:
   * fine tuning "gemma2_2b_en" on the dolly dataset
   * shows prompt completion before and after fine-tuning this model
   * it runs successfully in COLAB


# Get access to Gemma via your Kaggle account:
  * Log into your Kaggle account
  * Request access to Gemma models using your Kaggle account.  You can follow these instructions here: https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp
  * You need to wait for confirmation.  Note that this didn't take too long for me.
  * Create an API key in your Kaggle account you will need later.  You can follow these instructions here: https://christianjmills.com/posts/kaggle-obtain-api-key-tutorial/



# Ensure your Colab account can access Gemma:
  * Add the Kaggle API key into your COLAB secrets.  You can follow these instructions here: https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0



# Select an AI hardware accelerator
  * Select hardware options near the top right of your Colab notebook
  * I tested with A100 and it worked well.  Note that I have a Colab Pro subscription.


# Install required python packages

In [1]:
%%time
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.4 MB/s eta 0:00:00
CPU times: user 62.1 ms, sys: 8.27 ms, total: 70.4 ms
Wall time: 8.64 s


# Import required python packages

In [2]:
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
from IPython.display import Markdown
import textwrap
from google.colab import userdata
import json

# Configure this notebook
* set up KERAS parameters recommended by Google
* integrate KAGGLE API secret key

In [3]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" # Avoid memory fragmentation on JAX backend.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Retrieve the fine-tuning dataset

In [4]:
%%time
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
!pwd
!ls

--2024-09-27 16:41:07--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 13.35.210.77, 13.35.210.114, 13.35.210.61, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.77|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1727714467&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzcxNDQ2N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIw

# Define some useful functions used later
* display_chat() function

In [5]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

# Load the fine-tuning dataset
* loads the dataset into an array for use later
* it also formats each record into a specific string format

In [6]:
ft_data = []
with open("/content/databricks-dolly-15k.jsonl") as file: # For COLAB
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        ft_data.append(template.format(**features))

# Decide on how much fine-tuning data to use
* Often this is determined experimentally
* I've found at least 1000 data points suffice in general

In [7]:
data = ft_data[:1000]

# Load the Gemma model

In [8]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
# uncomment the following lines to "sample the softmax probabilities of the model"
#sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
#gemma_lm.compile(sampler=sampler)

CPU times: user 10.6 s, sys: 9.7 s, total: 20.3 s
Wall time: 51.7 s


# Have the model complete a "raw" prompt with no formatting


In [9]:
%%time
prompt = "What should I do on a trip to Europe?"
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 12s, sys: 1.61 s, total: 2min 13s
Wall time: 1min 2s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I do on a trip to Europe?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Europe for the first time in a few months. I'm going to be in Paris, London, and Amsterdam. I'm not sure what to do. I'm not a big fan of museums, but I'm not sure what else to do. I'm not a big fan of shopping, but I'm not sure what else to do. I'm not a big fan of nightlife, but I'm not sure what else to do. I'm not a big fan of food, but I'm not sure what else to do. I'm not a big fan of sports, but I'm not sure what else to do. I'm not a big fan of music, but I'm not sure what else to do. I'm not a big fan of movies, but I'm not sure what else to do. I'm not a big fan of TV, but I'm not sure what else to do. I'm not a big fan of books, but I'm not sure what else to do. I'm not a big fan of video games, but I'm not sure what else to do. I'm not a big fan of computers, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to</blockquote></font>

# Have the model complete a formatted prompt

In [10]:
# Ask a simple query this time using a specific template per the documentation
#
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>What should I do on a trip to Europe?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to</blockquote></font>

# Enable the model for fine-tuning

In [11]:
gemma_lm.backbone.enable_lora(rank=4)

# Fine-tune the model

In [12]:
%%time

# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 162s 59ms/step - loss: 0.8368 - sparse_categorical_accuracy: 0.5382
CPU times: user 4min 42s, sys: 11.7 s, total: 4min 54s
Wall time: 2min 42s


# Have the fine-tuned model complete a "raw" prompt

In [13]:
%%time
prompt = "What should I do on a trip to Europe?"
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 49.8 s, sys: 1.08 s, total: 50.9 s
Wall time: 51.4 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I do on a trip to Europe?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Europe for the first time in a few months. I'm going to be in Paris, Amsterdam, and London. I'm not sure what to do in each city. I'm looking for some suggestions.<br><br>[User 0002]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0003]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0004]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0005]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0006]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0007]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0008]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0009]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0010]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0011]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0012]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0013]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0014]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0015]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0016]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0017]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0018]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0019]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0020]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0021]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0022]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0023]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0024]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0025]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0026]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0027]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0028]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0029]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0030]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0031]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0032]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0033]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0034]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[User 0035]<br><br>I'm not sure what you mean by "what to do" in each city.<br><br>[</blockquote></font>

# Have the fine-tuned model complete a formatted prompt

In [14]:
%%time
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 1min 10s, sys: 643 ms, total: 1min 11s
Wall time: 41 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>What should I do on a trip to Europe?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>You should take a train to Paris, France.</blockquote></font>